# **Predicting Building Permit Issuance Times**
##                                                                                             ...and answering many questions!


**A Data Science Project**      
*by Aparna Shastry*

## Content
+ Introduction and Scope
+ Data
+ Cleaning the Data


## Introduction
A building permit is an official approval document issued by a governmental agency that allows you or your contractor to proceed with a construction or remodeling project on one's property. For more details click [here](https://www.thespruce.com/what-is-a-building-permit-1398344). Each city or county has its own office related to buildings, that can do multiple functions like issuing permits, inspecting buildings to enforce safety measures, modifying rules to accommodate needs of the growing population etc. For the city of San Francisco, permit issuing is taken care by [Permit Services wing of Department of Building Inspection](http://sfdbi.org/permit-services) (henceforth called DBI).
The delays in permit issuance pose serious problems to construction industries and later on real estate agencies.Read this [Trulia study](https://www.trulia.com/blog/trends/elasticity-2016/) and [Vancouver city article](https://biv.com/article/2014/11/city-building-permit-delays-costing-developers-tim).

### Scope of the Project

Primary objective of this Data Science Project is to predict the time delay window for the time in days between permit application and issuance for buildings in San Francisco city in California, USA. Since it is not possible to accurately predict the delay in resolution of days, the problem is limited to predicting if a permit will be issued in a week, or in 3 months or beyond 3 months.

Apart from this, we also draw some insights from the data to find out details that are of interest to the applicants.

## Data
Data for this project is available in San Francisco city open data portal. It is updated every Saturday.
1. Go to the link: [SF](https://data.sfgov.org/Housing-and-Buildings/Building-Permits/i98e-djp9/data) open portal. 
2. Click on Filter and "Add a Filter Condition".
3. A drop down menu appears.
4. Select, "Filed Date" and "is after".
5. I entered date as 12/31/2012, because I wanted to do analysis of last 4-5 years. I think most recent data is important in matters such as this, the city council policies could change, there might be new rules, new employers to expedite process etc. Old data may not be too useful in modeling.
6. I chose to download in CSV format because it is of the less than 100MB size and easy to load into notebook. If it gave issues, I might have chosen a different format.

The file as of Feb 25, 2018 (Sunday) has been downloaded and kept for easy access. Size is about 75MB

We will get to the specifics once we get initial glimpse of columns in the data.

In [1]:
import os
import pandas as pd
import numpy as np
import time
import datetime

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style(style='darkgrid')

In [2]:
%%time
# Read and make a copy for speed
sf = pd.read_csv('../data/Building_Permits.csv',low_memory=False)
sf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198900 entries, 0 to 198899
Data columns (total 43 columns):
Permit Number                             198900 non-null object
Permit Type                               198900 non-null int64
Permit Type Definition                    198900 non-null object
Permit Creation Date                      198900 non-null object
Block                                     198900 non-null object
Lot                                       198900 non-null object
Street Number                             198900 non-null int64
Street Number Suffix                      2216 non-null object
Street Name                               198900 non-null object
Street Suffix                             196132 non-null object
Unit                                      29479 non-null float64
Unit Suffix                               1961 non-null object
Description                               198610 non-null object
Current Status                            198900 n

Specifically, we will be trying to answer the following in the next parts:

+ **Data Cleaning:**  

   1) Which of these columns should be retained for further analysis? This is the first question to answer because eliminating obviously non-useful columns would save lot of time.        
   2) Which rows to retain? This will prevent some bias in the data or it might introduce. Need to be done cautiously.     
   3) How to interpret the records with zero or very small values for cost variables related to the construction?               
   4) What to do we do if dates map to Saturday or Sunday? A date having invalid numbers for month and day of the month?      
   5) Should we replace blanks in some columns where the valid value is 'Y'?    
   6) What do we do with blank cells in date columns?      
   7) What are we going to do with records with unknown locations?
         

In [3]:
# Conversion to datetime
import traceback
try :
    sf['Filed Date'] = pd.to_datetime(sf['Filed Date'],errors='coerce')
    sf['Issued Date'] = pd.to_datetime(sf['Issued Date'],errors='coerce')
    sf['Current Status Date'] = pd.to_datetime(sf['Current Status Date'],errors='coerce')
except :    
    traceback.print_exc()

# Keep a copy to reload
sfcpy = sf.copy()

In [4]:
# Sometimes when re-run is required, one can start from just here, to save time
sf = sfcpy.copy()

In [5]:
# Rename for brevity/readability
sf = sf.rename(columns =   {'Neighborhoods - Analysis Boundaries':'neighborhoods',
                            'Permit Type' : 'perm_typ',
                            'Permit Type Definition': 'perm_typ_def',
                            'Filed Date':'file_dt',
                            'Issued Date':'issue_dt',
                            'Permit Expiration Date' : 'perm_exp_dt',
                            'Current Status' : 'cur_st',
                            'Current Status Date' : 'cur_st_dt',
                            'Structural Notification':'strct_notif',
                            'Number of Existing Stories':'no_exist_stry',
                            'Number of Proposed Stories':'no_prop_stry',
                            'Fire Only Permit':'fire_only_permit',
                            'Estimated Cost':'est_cost',
                            'Revised Cost':'rev_cost',
                            'Existing Use':'exist_use',
                            'Proposed Use': 'prop_use',
                            'Plansets':'plansets',
                            'Existing Construction Type': 'exist_const_type',
                            'Existing Construction Type Description': 'exist_const_type_descr',
                            'Proposed Construction Type': 'prop_const_type',
                            'Proposed Construction Type Description': 'prop_const_type_descr',
                            'Site Permit':'site_permit',
                            'Supervisor District':'sup_dist',
                            'Location':'location'
                            })

In [6]:
sf.head()

,Permit Number,perm_typ,perm_typ_def,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,exist_const_type,exist_const_type_descr,prop_const_type,prop_const_type_descr,site_permit,sup_dist,neighborhoods,Zipcode,location,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,NaN,Ellis,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,NaN,Geary,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,NaN,Pacific,Av,...,1.0,constr type 1,1.0,constr type 1,NaN,3.0,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)",1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,NaN,Pacific,Av,...,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)",1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,NaN,Market,St,...,3.0,constr type 3,NaN,NaN,NaN,6.0,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)",144548169992


## Cleaning the Data

The Tricky part of Data Wrangling in my knowledge so far,   

a) Knowing what is present in the 'null' cells, is it NaN or simply ' '     
b) In the non-null cells, if the all values are meaningful       
c) Recognizing that even if a column has all non-null and meaningful values, the future updates to the column may have problems. Hence need to expect it and handle it      
d) See the data and think if certain value make sense for the business and decide to drop those which are not relevant

##### Answering the questions:
1) The following columns are retained for further analysis.

In [7]:
sfr = sf[['perm_typ','perm_typ_def','file_dt','issue_dt','cur_st','strct_notif','no_exist_stry','no_prop_stry',
          'fire_only_permit','est_cost','rev_cost','exist_use','prop_use','plansets','exist_const_type',
          'prop_const_type','site_permit','location']].copy()

2) Which rows to retain?       
We can eliminate some records based on current status.

In [8]:
sfr.cur_st.unique()

array(['expired', 'issued', 'withdrawn', 'complete', 'filed', 'approved',
       'reinstated', 'cancelled', 'appeal', 'suspend', 'revoked',
       'plancheck', 'disapproved', 'incomplete'], dtype=object)

+ We are sure of retaining rows where cur_st is issued or issue_dt column is a valid date.
+ We can retain rows corresponding to cur_st equal to complete,filed,approved,reinstated,appeal or plancheck irrespective of whether there is valid entry in issue_dt or not
+ We should drop rows for which cur_st is withdrawn, cancelled or disapproved and issue_dt is blank because they will never get issued. 
+ It is not clear what to do with suspend,revoked and incomplete. That can be decided with further exploration

In [9]:
# Check count of status of permits and under each status, how many are issued. 
sfr.groupby('cur_st')[['cur_st','issue_dt']].count()

,cur_st,issue_dt
cur_st,,
appeal,2,0
approved,733,8
cancelled,1536,1197
complete,97077,97059
disapproved,2,0
expired,1370,1370
filed,12043,0
incomplete,2,1
issued,83559,83559


In [10]:
# Drop withdrawn, disapproved and canceled which are not yet issued
sfr = sfr.loc[(((sfr['cur_st']!='withdrawn') & (sfr['cur_st'] !='cancelled') & (sfr['cur_st'] !='disapproved')) | sfr['issue_dt'].notnull()),:]

In [11]:
sfr[['file_dt','issue_dt','perm_typ']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 196806 entries, 0 to 198899
Data columns (total 3 columns):
file_dt     196806 non-null datetime64[ns]
issue_dt    183960 non-null datetime64[ns]
perm_typ    196806 non-null int64
dtypes: datetime64[ns](2), int64(1)
memory usage: 6.0 MB


Can we eliminate rows for which there are no issue dates?

Note that permit type and file dates have no NaNs, whereas issue date has a few thousands of NaNs, because at the time of data download, they were not yet issued. We decide to keep it as is at this stage, because dropping them would introduce survivorship bias in others. Imputation would still introduce bias.

In [12]:
sfr = sfr.assign(time_taken = (sfr['issue_dt'] - sfr['file_dt']).dt.days)

3) NaN or too small costs: Cost of the project is an essential part of the application according to [this post](http://www.herald-journal.com/housing/pages/older/permit.html). Examining if the cost columns have valid values.

In [13]:
print('Percentage of null values in revised cost:',round(100*sfr['rev_cost'].isnull().sum()/sfr['rev_cost'].isnull().count(),4))
print('Percentage of null values in estimated cost:',round(100*sfr['est_cost'].isnull().sum()/sfr['est_cost'].isnull().count(),4))

Percentage of null values in revised cost: 2.5497
Percentage of null values in estimated cost: 19.3282


In [14]:
print('Percentage of values < 10$ in estimated cost: {:.2f} and the same in revised cost is {:.2f}'
     .format(100*(sfr['est_cost'] < 10).sum()/(sfr['est_cost'] < 10).count(),100*(sfr['rev_cost'] < 10).sum()/(sfr['rev_cost'] < 10).count() ))

Percentage of values < 10$ in estimated cost: 8.55 and the same in revised cost is 27.94


Revised cost is most recent, and also has less null values compared to estimated cost. Hence, revised cost is more meaningful. However, since estimated cost has 20% null values, examining the time taken for those records might reveal something. Permit application without cost estimation might be delayed. The moment revised cost was entered, application might be processed. We would know that in the EDA part. For now, it is best to leave these alone. We could come back after cleaning some more columns.
Do not drop any rows based on Revised cost.

In [15]:
sfr['rev_cost'].fillna(sfr['rev_cost'].min(),inplace=True)

4) I would attribute it to typing mistake and make it previous or next day respectively. This may not be accurate, however it will not show a weekend in the EDA part.   

In [16]:
sfr['file_day'] = sfr['file_dt'].dt.weekday_name
sfr['issue_day'] = sfr['issue_dt'].dt.weekday_name
sfr.loc[sfr['file_day']=='Saturday','file_dt']  = sfr.loc[sfr['file_day']=='Saturday','file_dt'] - datetime.timedelta(1)
sfr.loc[sfr['file_day']=='Saturday','file_day'] = 'Friday'
sfr.loc[sfr['file_day']=='Sunday','file_dt'] =  sfr.loc[sfr['file_day']=='Sunday','file_dt'] + datetime.timedelta(1)
sfr.loc[sfr['file_day']=='Sunday','file_day'] = 'Monday'
sfr.loc[sfr['issue_day']=='Saturday','issue_dt'] = sfr.loc[sfr['issue_day']=='Saturday','issue_dt'] - datetime.timedelta(1)
sfr.loc[sfr['issue_day']=='Saturday','issue_day'] = 'Friday'
sfr.loc[sfr['issue_day']=='Sunday','issue_dt'] = sfr.loc[sfr['issue_day']=='Sunday','issue_dt'] + datetime.timedelta(1)
sfr.loc[sfr['issue_day']=='Sunday','issue_day'] = 'Monday'

5) In the application forms (both physical or online), normally the applicant is supposed to tick the option if applicable. Otherwise nothing needs to be done. Hence it is understandable that blanks mean not applicable, a "No".

In [17]:
# Fill na with N. because in building permit databases, this is ticked if yes and left blank if it is not applicable
sfr['fire_only_permit'].fillna('N',inplace=True)
sfr['site_permit'].fillna('N',inplace=True)
sfr['strct_notif'].fillna('N',inplace=True)

In [18]:
sfr['month'] = sfr['file_dt'].dt.month
sfr['year'] = sfr['file_dt'].dt.year

There is no need to check for validity in month or days because otherwise datatime conversion would have failed.

In [19]:
print('There is {}% NaN in plansets'.format(round(100*sfr[sfr.exist_const_type.isnull()]['plansets'].isnull().sum()/sfr.shape[0],2)))

There is 18.96% NaN in plansets


We let the NaN's in exist_const_type,prop_const_type,no_exist_stry,no_prop_stry, and plansets as they are because this is building permit application. If it is not mentioned, it is not applicable.

In [20]:
# Fill location unknowns with 0's to indicate, it is unknown
sfr.location.fillna('(0,0)',inplace=True)

#Convert to float
sfr.location = sfr.location.apply(lambda x: np.array([float((str(x).split('(')[1]).split(',')[0]),float((str(x).split('(')[1]).split(',')[1].split(')')[0])]))

I make a choice to drop the data after Sept 30th, 2017, and I do it in the data wrangling itself. I could have done it at EDA.
The reason for dropping is, many of the filed applications do not have issue dates. We retained it till here to know the issue dates for permits filed prior to that.

In [21]:
# Splitting into latitude and longitude
sfr = sfr.assign(latitude=sfr.location.apply(lambda x:x[0]))
sfr = sfr.assign(longitude=sfr.location.apply(lambda x: x[1]))
sfr.drop('location',axis=1,inplace=True)

In [22]:
# Replace nans with strings "Unknown"
sfr.exist_use.fillna('Unknown',inplace=True)
sfr.prop_use.fillna('Unknown',inplace=True)

In [23]:
sfr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 196806 entries, 0 to 198899
Data columns (total 24 columns):
perm_typ            196806 non-null int64
perm_typ_def        196806 non-null object
file_dt             196806 non-null datetime64[ns]
issue_dt            183960 non-null datetime64[ns]
cur_st              196806 non-null object
strct_notif         196806 non-null object
no_exist_stry       154195 non-null float64
no_prop_stry        154150 non-null float64
fire_only_permit    196806 non-null object
est_cost            158767 non-null float64
rev_cost            196806 non-null float64
exist_use           196806 non-null object
prop_use            196806 non-null object
plansets            159498 non-null float64
exist_const_type    153629 non-null float64
prop_const_type     153863 non-null float64
site_permit         196806 non-null object
time_taken          183960 non-null float64
file_day            196806 non-null object
issue_day           183960 non-null object
month 

In [24]:
# Store the dataframe to pass on to the next notebook
%store sfr

Stored 'sfr' (DataFrame)
